##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Simulation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/simulation"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/simulation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/simulation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/simulation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

installing cirq...
     |████████████████████████████████| 1.5 MB 6.7 MB/s 
     |████████████████████████████████| 380 kB 35.7 MB/s 
     |████████████████████████████████| 1.3 MB 43.2 MB/s 
installed cirq.


## Introduction to pure state simulation

Here is a simple circuit:

In [2]:
import cirq

q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)

def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0, q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')

circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


We can simulate this by creating a ``cirq.Simulator`` and
passing the circuit into its ``run`` method:

In [3]:
from cirq import Simulator
simulator = Simulator()
result = simulator.run(circuit)

print(result)

q0=1
q1=1


In [4]:
result = simulator.run(circuit)

print(result)

q0=0
q1=1


The simulator is designed to mimic what running a program
on a quantum computer is actually like.  

### Accessing the state vector


In [7]:
import numpy as np
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))    
result = simulator.simulate(circuit, qubit_order=[q0, q1])

print(np.around(result.final_state_vector, 3))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


### Expectation values


In [8]:
XX_obs = cirq.X(q0) * cirq.X(q1)
ZZ_obs = cirq.Z(q0) * cirq.Z(q1)
ev_list = simulator.simulate_expectation_values(
    cirq.Circuit(basic_circuit(False)),
    observables=[XX_obs, ZZ_obs],
)
print(ev_list)

[(1+0j), 0j]


`simulate_expectation_values()` returns a list of
expectation values, one for each observable provided.

## Qubit and Amplitude Ordering



In [9]:
outside = [1, 10]
inside = [1, 2]
print(np.kron(outside, inside))

[ 1  2 10 20]


In [10]:
i = 0
for first in [0, 1]:
    for second in [0, 1]:
        print('amps[{}] is for first={}, second={}'.format(i, first, second))
        i += 1

amps[0] is for first=0, second=0
amps[1] is for first=0, second=1
amps[2] is for first=1, second=0
amps[3] is for first=1, second=1


We can check that this is in fact the ordering with a
circuit that flips one qubit out of two:

In [11]:
q_stay = cirq.NamedQubit('q_stay')
q_flip = cirq.NamedQubit('q_flip')
c = cirq.Circuit(cirq.X(q_flip))

# first qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_flip, q_stay])
print(abs(result.final_state_vector).round(3))

[0. 0. 1. 0.]


In [12]:
# second qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_stay, q_flip])
print(abs(result.final_state_vector).round(3))

[0. 1. 0. 0.]


## Stepping through a circuit



In [13]:
circuit = cirq.Circuit()
circuit.append(basic_circuit())    
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


The object returned by the `moment_steps` iterator is a
`StepResult`. This object has the state along with any
measurements that occurred **during** that step (so does
not include measurement results from previous `Moments`).

In addition, the `StepResult` contains `set_state_vector()`,
which can be used to set the `state`. One can pass a valid
full state to this method by passing a numpy array. Or,
alternatively, one can pass an integer, and then the state
will be set to lie entirely in the computation basis state
for the binary expansion of the passed integer.

### Alternate stepping behavior



In [14]:
chunks = [cirq.Circuit(moment) for moment in basic_circuit()]
next_state = 0  # represents the all-zero state
for i, chunk in enumerate(chunks):
    result = simulator.simulate(chunk, initial_state=next_state)
    next_state = result.final_state_vector
    print(f'state at step {i}: {np.around(next_state, 3)}')

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+1.j 0.+0.j]


The added cost of passing state vectors around like this
is nontrivial; for this reason, this workaround should
only be used with simulators that do not support
`simulate_moment_steps`.

## Parameterized values and studies



In [15]:
import sympy
rot_w_gate = cirq.X**sympy.Symbol('x')
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
for y in range(5):
    resolver = cirq.ParamResolver({'x': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print(np.round(result.final_state_vector, 2))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [16]:
resolvers = [cirq.ParamResolver({'x': y / 2.0}) for y in range(3)]
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
circuit.append([cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')])
results = simulator.run_sweep(program=circuit,
                              params=resolvers,
                              repetitions=2)
for result in results:
    print(result)

q0=00
q1=00
q0=11
q1=11
q0=11
q1=11


Above we see that assigning different values to gate parameters yields
different results for each trial in the sweep, and that each trial is repeated
`repetitions` times.

## Mixed state simulations

In addition to pure state simulation, Cirq also supports
simulation of mixed states. 

Even though this simulator is not as efficient as the pure state simulators, they allow for a larger class of noisy circuits to be run as well as keeping track of the simulation's density matrix. 

In [17]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 64, 1: 36})


In [18]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit)
print(np.around(result.final_density_matrix, 3))

[[0.6  +0.j 0.447+0.j]
 [0.447+0.j 0.4  +0.j]]


We see that we have access to the density matrix at the
end of the simulation via `final_density_matrix`.

## External simulators

There are a few high-performance circuit simulators which
provide an interface for simulating Cirq `Circuit`s.
These projects are listed below, along with their PyPI package
name and a description of simulator methods that they support.

**Note:** In general, these simulators are optimized for
specific use cases. Before choosing a simulator, make sure it
supports the behavior that you need!

| Project name | PyPI package | Description |
| --- | --- | --- |
| [qsim](https://github.com/quantumlib/qsim) | qsimcirq | Implements `cirq.SimulatesAmplitudes` and `cirq.SimulatesFinalState`. Recommended for deep circuits with up to 30 qubits (consumes 8GB RAM). Larger circuits are possible, but RAM usage doubles with each additional qubit. |
| [qsimh](https://github.com/quantumlib/qsim/blob/master/qsimcirq/qsimh_simulator.py) | qsimcirq | Implements `cirq.SimulatesAmplitudes`. Intended for heavy parallelization across several computers; Cirq users should generally prefer qsim. |
| [qFlex](https://github.com/ngnrsaa/qflex) | qflexcirq | Implements `cirq.SimulatesAmplitudes`. Recommended for shallow / low-entanglement circuits with more than 30 qubits. RAM usage is highly dependent on the number of two-qubit gates in the circuit. |
| [quimb](https://quimb.readthedocs.io/en/latest/) | quimb | Cirq-to-quimb translation layer provided in `contrib/quimb`. In addition to circuit simulation, this allows the use of quimb circuit-analysis tools on Cirq circuits. |


# Copyright 2020 The Cirq Developers 

This code is licensed under the Apache License, Version 
2.0. You may obtain a copy of this license in the 
LICENSE.txt file in the root directory of this source tree 
or at http://www.apache.org/licenses/LICENSE-2.0.


Program executed by Bhadale IT in Google Colab 
(https://www.bhadaleit.com).
For more details on the code and tutorials visit
https://quantumai.google/cirq/ website